In [1]:
!pip install timm

In [3]:
import os
import gc
import cv2
import random
import time
import copy
from PIL import Image

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision.transforms as T

import joblib
from tqdm import tqdm
from collections import defaultdict

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split

import timm

from colorama import Fore, Back, Style
g_ = Fore.GREEN
c_ = Fore.CYAN
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

In [4]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret('wandb_api')
    wandb.login(key=api_key)
    anony=None
except:
    anony = 'must'

In [6]:
ROOT_DIR = '../input/landmark-retrieval-2021'
TRAIN_DIR = '../input/landmark-retrieval-2021/train'
TEST_DIR = '../input/landmark-retrieval-2021/test'

In [7]:
CONFIG = dict(
    seed = 42,
    model_name = 'tf_mobilenetv3_small_100',
    train_batch_size = 256,
    valid_batch_size = 512,
    img_size = 256,
    epochs = 6,
    learning_rate = 5e-4,
    min_lr = 1e-6,
    T_max = 100,
    T_0 = 25,
    weight_decay=1e-6,
    n_accumulate = 1,
    num_classes = 81313,
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
    competition = 'Google',
    wandb_kernel = 'deb',
    n_fold = 5
)

In [9]:
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    os.environ['PYTHONHASHSEED'] = str(seed)  # set a fixed value for the 'hash' seed
    
set_seed(CONFIG['seed'])

# Read the Data

In [10]:
def get_train_file_path(id):
    return f'{TRAIN_DIR}/{id[0]}/{id[1]}/{id[2]}/{id}.jpg'

In [13]:
df = pd.read_csv(f'{ROOT_DIR}/train.csv')

le = LabelEncoder()
df['landmark_id'] = le.fit_transform(df['landmark_id'])

df['file_path'] = df['id'].apply(get_train_file_path)
df.head()

# Visualize Image

In [ ]:
run = wandb.init(project='GLRet2021',
                 config=CONFIG,
                 job_type='Visualizaion',
                 anonymous='must')

In [ ]:
preview_table = wandb.Table(columns=['Id', 'Image', 'Landmark'])
tmp_df = df.sample(3000, random_state=CONFIG['seHorizontalFlipndex(drop=True)
for i in tqdm(range(len(tmp_df))):
    row = tmp_df.loc[i]
    img = Image.open(row['file_path'])
    preview_table.add_data(row['id'], wandb.Image(img), row['landmark_id'])
    
wandb.log({'Visualization':preview_table})
run.finish()

In [ ]:
df_train, df_valid = train_test_split(df, test_size=0.3, stratify=df['landmark_id'], shuffle=True, random_state=CONFIG['seed'])

In [15]:
df_train, df_test = train_test_split(df, test_size=0.4, stratify=df['landmark_id'],
                                     shuffle=True, random_state=CONFIG['seed'])  # Train셋, Test셋 나누기
df_valid, df_test = train_test_split(df_test, test_size=0.5, shuffle=True,
                                     random_state=CONFIG['seed'])  # Valid셋과 Test셋 나누기

# Dataset

In [17]:
class LandmarkDataset(Dataset):
    def __init__(self, root_dir, df, transforms=True):
        self.root_dir = root_dir
        self.df = df
        self.file_name = df['file_path'].values
        self.labels = df['landmark_id'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_name[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        labels = self.labels[index]
        if self.transforms:
            img = self.transforms(image=img)['image']
        
        return img, labels

# Data Augmentations

In [18]:
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

data_transforms = {
    'train' : A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.HorizontalFlip(),
        A.CoarseDropout(),
        A.Normalize(mean=MEAN, std=STD, max_pixel_value=255.0, p=1.0),
        ToTensorV2()
    ], p=1.0),
    
    'valid' : A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(mean=MEAN, std=STD, max_pixel_value=255.0, p=1.0),
        ToTensorV2()
    ], p=1.0)
}

# Model

In [19]:
class LandmarkModel(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super(LandmarkModel, self).__init__()
        self.model = timm.create_model(model_name=model_name, pretrained=pretrained)
        self.in_features = self.model.classifier.in_features
        self.model.reset_classifier(0)
        self.fc = nn.Linear(self.in_features, CONFIG['num_classes'])
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, images):
        features = self.model(images)
        features = self.dropout(features)
        output = self.fc(features)
        return output
    
model = LandmarkModel(CONFIG['model_name'])
model.to(CONFIG['device'])

In [ ]:
class LandmarkModel(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super(LandmarkModel, self).__init__()
        self.model = timm.create_model(model_name=model_name, pretrained=pretrained)
        self.in_features = self.model.classifier.in_features
        self.model.reset_classifier(0)
        self.fc = nn.Linear(self.in_features, CONFIG['num_classes'])
        # self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        features = self.model(x)
        # features = self.dropout(features)
        output = self.fc(features)
        return output
    
    def extract_features(self, x):
        features = self.model(x)
        # features = self.dropout(features)
        return features
    
model = LandmarkModel(CONFIG['model_name'])
model.to(CONFIG['device'])

In [20]:
# Loss Function

def criterion(outputs, targets):
    return nn.CrossEntropyLoss()(outputs, targets)

# Training 

In [31]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    scaler = amp.GradScaler()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, (images, labels) in bar:
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.long)
        
        batch_size = images.size(0)
        
        with amp.autocast(enabled=True):
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss = loss / CONFIG['n_accumulate']
            
        scaler.scale(loss).backward()
        
        if (step+1) % CONFIG['n_accumulate'] == 0:
            scaler.step(optimizer)
            scaler.update()
            
            optimizer.zero_grad()
            
            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss, LR=optimizer.param_groups[0]['lr'])
        
    gc.collect()
    
    return epoch_loss

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    scaler = amp.GradScaler()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, (images, labels) in bar:         
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.long)
        
        batch_size = images.size(0)
        
        with amp.autocast(enabled=True):
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss = loss / CONFIG['n_accumulate']
            
        scaler.scale(loss).backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            scaler.step(optimizer)
            scaler.update()

            # zero the parameter gradients
            # optimizer.zero_grad()
            for p in model.parameters():
                p.grad = None

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss/dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss

# Validation

In [22]:
@torch.no_grad()
def valid_one_epoch(model, model_name, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    TARGETS = []
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, (images, labels) in bar:
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.long)
        
        batch_size = images.size(0)
        
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        PREDS.append(preds.view(-1).cpu().detach().numpy())
        TARGETS.append(labels.view(-1).cpu().detach().numpy())
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss, LR=optimizer.param_groups[0]['lr'])
    
    TARGETS = np.concatenate(TARGETS)
    PREDS = np.concatenate(PREDS)
    val_acc = accuracy_score(TARGETS, PREDS)
    gc.collect()
    
    return epoch_loss, val_acc

In [ ]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    TARGETS = []
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, (images, labels) in bar:        
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.long)
        
        batch_size = images.size(0)
        
        outputs = model(images)
        _, preds = torch.max(outputs,1)
        loss = criterion(outputs, labels)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss/dataset_size
        
        PREDS.append(preds.view(-1).cpu().detach().numpy())
        TARGETS.append(labels.view(-1).cpu().detach().numpy())
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])   
    
    TARGETS = np.concatenate(TARGETS)
    PREDS = np.concatenate(PREDS)
    val_acc = accuracy_score(TARGETS, PREDS)
    gc.collect()
    
    return epoch_loss, val_acc

# Run Training

In [24]:
def run_training(model, optimizer, scheduler, device, num_epochs):
    wandb.watch(model, log_freq=100)
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_acc = 0
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs+1):
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler,
                                           dataloader=train_loader, device=CONFIG['device'], epoch=epoch)
        val_epoch_loss, val_epoch_acc = valid_one_epoch(model, dataloader=valid_loader, 
                                                        device=CONFIG['device'], epoch=epoch)
        
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Valid Acc'].append(val_epoch_acc)
        
        wandb.log({'Train Loss' : train_epoch_loss})
        wandb.log({'Valid Loss' : val_epoch_loss})
        wandb.log({'Valid Acc' : val_epoch_acc})
        
        print(f'Valid Accuracy : {val_epoch_acc}')
        
        if val_epoch_acc >= best_epoch_acc:
            print(f'{c_}Validation Acc Improved({best_epoch_acc} --> {val_epoch_acc})')
            best_epoch_acc = val_epoch_acc
            run.summary['Best Accuracy'] = best_epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f'Epoch{epoch}.bin'
            torch.save(model.state_dict(), PATH)
            wandb.save(PATH)
            print(f'Model Saved{sr_}')
            
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best ACC: {:.4f}".format(best_epoch_acc))
    
    model.load_state_dict(best_model_wts)
    
    return model, history

In [26]:
# DataLoader
def prepare_loader():
    train_dataset = LandmarkDataset(TRAIN_DIR, df_train, transforms=data_transforms['train'])
    valid_dataset = LandmarkDataset(TRAIN_DIR, df_valid, transforms=data_transforms['valid'])
    
    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], shuffle=True, pin_memory=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

train_loader, valid_loader = prepare_loader()

In [27]:
optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG['T_max'], eta_min=CONFIG['min_lr'])

In [28]:
run = wandb.init(project='GLRet2021', 
                 config=CONFIG,
                 job_type='Train',
                 anonymous='must')

In [32]:
mode, history = run_training(model, optimizer, scheduler,
                            device=CONFIG['device'], num_epochs=CONFIG['epochs'])